# Connexion_France_Travail_ADZUNA.ipynb

Ce script a pour objectif d'extraire les offres d'emploi mises à disposition par l'API de France Travail et de l'API ADZUNA et de les stocker dans un fichier (CSV ou Excel) en local.

Comment ?
1. Sur la base de critères spécifiques (mots clés, localisation, etc...), 
    - lancement d'une requête pour obtenir les offres d'emploi correspondantes via l'API France Travail
    - lancement d'une requête pour obtenir les offres d'emploi correspondantes via l'API Adzuna
2. Une fois les offres trouvées, vérification et suppression des doublons.
3. Une sauvegarde en local des offres sont stockées dans un fichier (CSV ou Excel).

Les URL (FRANCE TRAVAIL) utiles sont :
- https://francetravail.io/data/api/offres-emploi
- https://francetravail.io/data/api/offres-emploi/documentation#/

Les URL (ADZUNA) utiles sont :
- https://developer.adzuna.com/overview
- https://developer.adzuna.com/docs/search
- https://developer.adzuna.com/activedocs#!/adzuna/search
- https://developer.adzuna.com/overview
- https://www.adzuna.fr/details/5376850320?utm_medium=api&utm_source=6d1ef246

## Imports

In [33]:
import http.client
import requests
import json
import pandas as pd
import os
import hashlib

## Procédure

### Configuration

In [34]:
##################  VARIABLES  ##################
# France Travail
FT_CLIENT_ID = os.environ.get("FT_CLIENT_ID")
FT_CLIENT_SECRET = os.environ.get("FT_CLIENT_SECRET")
FT_SCOPE = os.environ.get("FT_SCOPE")

# Adzuna
ADZUNA_CLIENT_ID = os.environ.get("ADZUNA_CLIENT_ID")
ADZUNA_CLIENT_SECRET = os.environ.get("ADZUNA_CLIENT_SECRET")

### Paramètres de recherche

In [35]:
# ---------------------------
# PARAMETRES
# ---------------------------

# Paramètres de recherche
JOB_QUERY = "data analyst"
COMMUNE = "78300"
DISTANCE = 100000

# Nombre d'annonces par page requise
BLOC_PAGINATION = 50

# Nombre de pages max
MAX_PAGES = 20   # Limiter le nombre de pages récupérées

### Authentification France Travail

In [36]:
# ---------------------------
# AUTH FRANCE TRAVAIL
# ---------------------------
def get_ft_token():
    url = "https://entreprise.pole-emploi.fr/connexion/oauth2/access_token?realm=/partenaire"
    data = {
        "grant_type": "client_credentials",
        "client_id": FT_CLIENT_ID,
        "client_secret": FT_CLIENT_SECRET,
        "scope": FT_SCOPE,
    }
    r = requests.post(url, data=data)
    r.raise_for_status()
    return r.json()["access_token"]

### Lancement requête API France Travail

In [37]:
# ---------------------------
# API CALL FRANCE TRAVAIL
# ---------------------------
def fetch_france_travail_jobs(token, max_pages=MAX_PAGES):
    headers = {"Authorization": f"Bearer {token}"}
    all_jobs = []
    b_stop_criteria = False
    
    for page in range(1, max_pages + 1):
        if b_stop_criteria == False:    
            url = f"https://api.francetravail.io/partenaire/offresdemploi/v2/offres/search"
            params = {
                "motsCles": JOB_QUERY,
                "commune": COMMUNE,
                "distance" : DISTANCE,
                "range": f"{(page-1)*BLOC_PAGINATION}-{page*BLOC_PAGINATION-1}"  # pagination par blocs de 50
            }
            r = requests.get(url, headers=headers, params=params)
            r.raise_for_status()
            data = r.json()
            offres = data.get("resultats", [])
                
            for o in offres:
                all_jobs.append({
                    "Source": "France Travail",
                    "id":o.get("id"),    
                    "Titre": o.get("intitule"),
                    "Description": o.get("description"),
                    "Entreprise": o.get("entreprise", {}).get("nom"),
                    "Lieu": o.get("lieuTravail", {}).get("libelle"),
                    "Latitude": o.get("lieuTravail", {}).get("latitude"),
                    "Longitude": o.get("lieuTravail", {}).get("longitude"),
                    "typeContratLibelle": o.get("typeContratLibelle"),
                    "Date_creation": o.get("dateCreation"),
                    "URL": o.get("origineOffre").get("urlOrigine") if o.get("origineOffre") is not None else "None",
                    "Secteur_activites": o.get("secteurActiviteLibelle"),
                })
            # print(len(offres))

            # Si le nombre d'offres est inférieur au nombre max d'offre par pages, c'est un signe qu'il n'y a plus d'offres à extraire après la page actuelle.
            if len(offres) < BLOC_PAGINATION:
                b_stop_criteria = True
            
    return all_jobs

### Lancement de requête Adzuna

In [38]:
# ---------------------------
# API CALL ADZUNA
# ---------------------------
def fetch_adzuna_jobs(max_pages=MAX_PAGES):
    headers = {"Accept": "application/json"}
    all_jobs = []
    b_stop_criteria = False
    
    for page in range(1, max_pages + 1):
        if b_stop_criteria == False:    
            url = f"https://api.adzuna.com/v1/api/jobs/fr/search/{page}"
            params = {
                "app_id" : ADZUNA_CLIENT_ID,
                "app_key" : ADZUNA_CLIENT_SECRET,
                "title_only": JOB_QUERY,
                "where": COMMUNE,
                "results_per_page" : BLOC_PAGINATION,
                "distance" : DISTANCE
            }
            r = requests.get(url,params=params)
            r.raise_for_status()
            data = r.json()
            offres = data.get("results")
                
            # print(len(offres))

            for o in offres:
                all_jobs.append({
                    "Source": "Adzuna",
                    "id" : o.get("id"),
                    "Titre" : o.get("title"),
                    "Description" : o.get("description"),
                    "Entreprise": o.get("company").get("display_name") if o.get("company") is not None else "None",
                    "Lieu" : o.get("location").get("display_name") if o.get("location") is not None else "None",  
                    "Latitude" : o.get("latitude"),
                    "Longitude" : o.get("longitude"),
                    "typeContratLibelle" : o.get("contract_type"),                    
                    "Date_creation" : o.get("created"),
                    "URL" : o.get("redirect_url"),
                    "Secteur_activites" : o.get("category").get("label") if o.get("category") is not None else "None",
                })
                
            # Si le nombre d'offres est inférieur au nombre max d'offre par pages, c'est un signe qu'il n'y a plus d'offres à extraire après la page actuelle.
            if len(offres) < BLOC_PAGINATION:
                b_stop_criteria = True
            
    return all_jobs

### Déduplication

In [39]:
# ---------------------------
# DÉDUPLICATION
# ---------------------------
def deduplicate(jobs):
    seen = set()
    deduped = []
    for job in jobs:
        key_str = f"{job['Titre']}_{job['Entreprise']}_{job['Latitude']}_{job['Longitude']}_{job['Date_creation']}"
        key = hashlib.md5(key_str.encode()).hexdigest()
        if key not in seen:
            seen.add(key)
            deduped.append(job)
    return deduped

### Procédure principale

In [40]:
# ---------------------------
# MAIN
# ---------------------------
if __name__ == "__main__":
    print("Authentification France Travail...")
    token = get_ft_token()

    print("Récupération des offres France Travail...")
    ft_jobs = fetch_france_travail_jobs(token)

    print("Récupération des offres Adzuna...")
    adzuna_jobs = fetch_adzuna_jobs()

    print("Fusion et déduplication...")
    all_jobs = ft_jobs + adzuna_jobs

    print(f"Nombre d'offres d'emploi avant déduplication : {len(all_jobs)}")
    jobs_clean = deduplicate(all_jobs)
    print(f"Nombre d'offres d'emploi après déduplication : {len(jobs_clean)}")

    print("Affichage Extract offres France Travail...")
    df = pd.DataFrame(jobs_clean)
    display(df.shape)
    display(df.head())

    # Export vers CSV
    path = "../data/raw_data/"
    file_name = "offres_emploi_france_travail_adzuna.csv"
    
    df.to_csv(path + file_name, index=False, encoding="utf-8")
    print(f"{len(jobs_clean)} offres uniques exportées dans {file_name} ✅")

Authentification France Travail...
Récupération des offres France Travail...
Récupération des offres Adzuna...
Fusion et déduplication...
Nombre d'offres d'emploi avant déduplication : 1182
Nombre d'offres d'emploi après déduplication : 1180
Affichage Extract offres France Travail...


(1180, 12)

,Source,id,Titre,Description,Entreprise,Lieu,Latitude,Longitude,typeContratLibelle,Date_creation,URL,Secteur_activites
0,France Travail,197NKZD,Consultant Data confirmé - Data Analyst H/F,Nous recherchons un(e) Consultant(e) Data conf...,EFFIDIC,49 - TRELAZE,47.445931,-0.466974,CDI,2025-09-09T09:24:40.097Z,https://candidat.francetravail.fr/offres/reche...,Conseil en systèmes et logiciels informatiques
1,France Travail,197PLRS,Data Analyst (h/f),"Adecco recherche pour l'un de ses clients, une...",ADECCO FRANCE,13 - Marignane,43.407483,5.255358,Intérim - 12 Mois,2025-09-09T16:27:22.779Z,https://candidat.francetravail.fr/offres/reche...,Activités des agences de travail temporaire
2,France Travail,197FWLH,Data analyst senior (H/F),Data Analyst Senior F/H - CDI - La Défense\n\n...,TINGARI,92 - PUTEAUX,48.884019,2.237979,CDI,2025-09-02T11:49:20.769Z,https://candidat.francetravail.fr/offres/reche...,Conseil pour les affaires et autres conseils d...
3,France Travail,197GFLX,Data analyst (F/H),"Directement rattaché au DSI , vous serez un él...",EXPECTRA,34 - Montpellier,43.626378,3.882533,CDI,2025-09-02T14:41:39.561Z,https://candidat.francetravail.fr/offres/reche...,Activités des agences de travail temporaire
4,France Travail,197DQJQ,Data Analyst (H/F),"Le Groupe WATT&CO, acteur reconnu depuis 15 an...",WATT & CO INGENIERIE,81 - MAZAMET,43.489968,2.376418,CDI,2025-09-01T14:10:09.118Z,https://candidat.francetravail.fr/offres/reche...,"Ingénierie, études techniques"


1180 offres uniques exportées dans offres_emploi_france_travail_adzuna.csv ✅
